# Python Scripts for Facebook Data Discover


# I. Get all posts 

In [280]:
# Import necessary library
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import facebook as facebook
from datetime import datetime
import numpy as np

In [277]:
pd.set_option('display.max_columns', None)

In [281]:
# Connect Facebook with Consulate token
graph = facebook.GraphAPI(access_token="EAABnlB7bWhcBABde3nVXzemP41sx4mnAua62zr7vQZBkP4cfZAR0HEZAIesfJSsTjZBzCHZBsrFDSDRDMaKjGBgtMyjUc5wcS7iZC5gUNAHmJoZCcSO9tJ3yKUA6lUhxifvGXTRbOie58jn3WL8j2jtkydlf282TSHuMcIAW7PUaP6PgIWzz62ut0wLXo8Yf7QZD", version="3.1")

In [283]:
# Crawl posts and import to dataframe
allposts = graph.get_all_connections(id='165419643537532', connection_name ='posts', fields ='created_time, id, message, status_type, permalink_url, full_picture, message_tags, shares, is_popular')
list_allposts = list(allposts)
df = pd.DataFrame(list_allposts)

In [9]:
# Get reactions function
def get_post_reactions(row):
    item_reaction = graph.get_connections(id = row["id"], connection_name= "insights", metric="post_reactions_by_type_total", period = "lifetime")
    if "like" in item_reaction['data'][0]['values'][0]['value']:
        row["likes"] = item_reaction['data'][0]['values'][0]['value']['like']
    if "love" in item_reaction['data'][0]['values'][0]['value']:
        row["love"] = item_reaction['data'][0]['values'][0]['value']['love']
    if "wow" in item_reaction['data'][0]['values'][0]['value']:
        row["wow"] = item_reaction['data'][0]['values'][0]['value']['wow']
    if "haha" in item_reaction['data'][0]['values'][0]['value']:
        row["haha"] = item_reaction['data'][0]['values'][0]['value']['haha']
    if "sorry" in item_reaction['data'][0]['values'][0]['value']:
        row["sorry"] = item_reaction['data'][0]['values'][0]['value']['sorry']
    if "anger" in item_reaction['data'][0]['values'][0]['value']:
        row["anger"] = item_reaction['data'][0]['values'][0]['value']['anger']
    return row

In [156]:
# Get number of share
def get_share(row):
    if(type(row["shares"]) == dict):
        row["shares"] = row["shares"]["count"]
    if(np.isnan(row["shares"])):
        row["shares"] = np.NAN
    return row

In [294]:
# Get tags function
def get_tags(row):
    tags = []
    for tag in row['message_tags']:
        tags.append(tag['name'])
        row['message_tags'] = tags
    return row

In [320]:
def get_list_comment(row):
    comments = []
    for comment in row['comments']:
        comments.append(comment['message'])
        row['comments'] = comments
        #row['comments_count'] = len(comments)
    return row

In [292]:
# Get tags on dataframe
def get_tags_df(df_input):
    df_input[~df_input['message_tags'].isna()] = df_input[~df_input['message_tags'].isna()].apply(get_tags, axis=1)
    return df_input

In [346]:
# Get comments
def get_comments(row):
    comments_array = []
    comments = graph.get_all_connections(id=row["id"], connection_name = "comments", fields = "message")
    list_allcomments = list(comments)
    for comment in list_allcomments:
        comments_array.append(comment['message'])
        row['comments'] = comments_array
        row['comments_count'] = len(comments_array)
    return row

In [354]:
# Get actions and impression
def get_actions_impression(row): 
    item_action = graph.get_connections(id = row["id"], connection_name = "insights", metric='post_impressions, post_clicks_by_type_unique', date_preset='yesterday', period='lifetime', show_description_from_api_doc=False)
    if(row['status_type'] == 'added_video'):
        row["impression"] = item_action['data'][0]['values'][0]['value']
        row['video_play'] = item_action['data'][1]['values'][0]['value']['video play']
        row['link_clicks'] = item_action['data'][1]['values'][0]['value']['link clicks']
        row['other_clicks'] = item_action['data'][1]['values'][0]['value']['other clicks']
    if(row['status_type'] == 'added_photos' or row['status_type'] == 'mobile_status_update'):
        row["impression"] = item_action['data'][0]['values'][0]['value']
        row['photo_view'] = item_action['data'][1]['values'][0]['value']['photo view']
        if 'link clicks' in item_action['data'][1]['values'][0]['value']:
            row['link_clicks'] = item_action['data'][1]['values'][0]['value']['link clicks']
        row['other_clicks'] = item_action['data'][1]['values'][0]['value']['other clicks']
    return row
    


In [284]:
df

,created_time,id,message,status_type,permalink_url,full_picture,message_tags,shares,is_popular
0,2021-07-11T12:33:27+0000,165419643537532_4142798599132930,🇺🇸🏀 #Bạncóbiết Đội tuyển bóng rổ nam và nữ quố...,added_photos,https://www.facebook.com/165419643537532/posts...,https://scontent.fhan2-2.fna.fbcdn.net/v/t39.3...,"[{'id': '175644755921764', 'name': '#Bạncóbiết...",{'count': 3},False
1,2021-07-11T09:29:49+0000,165419643537532_4142790775800379,"📈 Hôm nay, ngày 11 tháng 7, là Ngày Dân số Th...",added_photos,https://www.facebook.com/165419643537532/posts...,https://scontent.fhan2-2.fna.fbcdn.net/v/t39.3...,"[{'id': '130104877168682', 'name': '#DYK', 'of...",{'count': 10},False
2,2021-07-10T12:06:41+0000,165419643537532_399386351491560,NaN,created_event,https://www.facebook.com/events/399386351491560/,https://scontent.fhan2-4.fna.fbcdn.net/v/t39.3...,NaN,NaN,False
3,2021-07-10T01:00:41+0000,165419643537532_4139411036138353,💉✈️ 𝟐 𝐭𝐫𝐢𝐞̣̂𝐮 𝐥𝐢𝐞̂̀𝐮 𝐯𝐚̆́𝐜 𝐱𝐢𝐧 #𝐌𝐨𝐝𝐞𝐫𝐧𝐚 𝐝𝐨 𝐇𝐨𝐚...,added_photos,https://www.facebook.com/165419643537532/posts...,https://scontent.fhan2-3.fna.fbcdn.net/v/t39.3...,"[{'id': '2682795158431299', 'name': '#𝐌𝐨𝐝𝐞𝐫𝐧𝐚'...",{'count': 1631},False
4,2021-07-09T07:01:23+0000,165419643537532_4137169826362474,"📣 Vào ngày 9 tháng 7 năm 2021, các cơ quan chứ...",added_photos,https://www.facebook.com/165419643537532/posts...,https://scontent.fhan2-4.fna.fbcdn.net/v/t39.3...,[],{'count': 54},False
...,...,...,...,...,...,...,...,...,...
6799,2011-10-03T08:53:12+0000,165419643537532_176095189136644,"On September 23, to mark a World Heart Day and...",added_photos,https://www.facebook.com/USConsulateHCMC/photo...,https://scontent.fhan2-2.fna.fbcdn.net/v/t1.18...,NaN,NaN,False
6800,2011-09-30T15:05:30+0000,165419643537532_175001792579317,NaN,added_photos,https://www.facebook.com/USConsulateHCMC/photo...,https://scontent.fhan2-4.fna.fbcdn.net/v/t1.18...,NaN,NaN,False
6801,2011-09-30T06:29:49+0000,165419643537532_174882175924612,NaN,added_photos,https://www.facebook.com/USConsulateHCMC/photo...,https://scontent.fhan2-3.fna.fbcdn.net/v/t1.18...,NaN,NaN,False
6802,2011-09-28T03:15:16+0000,165419643537532_174031516009678,The American Center - Grand Opening September ...,mobile_status_update,https://www.facebook.com/165419643537532/posts...,NaN,NaN,NaN,False


In [302]:
df_out = df.apply(get_share, axis=1)
df_out = df_out.apply(get_comments, axis=1)
df_out[~df_out['message_tags'].isna()] = df_out[~df_out['message_tags'].isna()].apply(get_tags, axis=1)

In [359]:
df_out.loc[0:20].apply(get_post_reactions, axis=1)
 

,anger,comments,comments_count,created_time,full_picture,haha,id,is_popular,likes,love,message,message_tags,permalink_url,shares,sorry,status_type,wow
0,NaN,"[U.S.A NUMBER ONE!, wow wow wow 🤩 , Đứng nghiê...",7.0,2021-07-11T12:33:27+0000,https://scontent.fhan2-2.fna.fbcdn.net/v/t39.3...,NaN,165419643537532_4142798599132930,False,193.0,20.0,🇺🇸🏀 #Bạncóbiết Đội tuyển bóng rổ nam và nữ quố...,"[#Bạncóbiết, #DYK]",https://www.facebook.com/165419643537532/posts...,3.0,NaN,added_photos,2.0
1,NaN,"[, We need more vigara...I think it alway work...",2.0,2021-07-11T09:29:49+0000,https://scontent.fhan2-2.fna.fbcdn.net/v/t39.3...,NaN,165419643537532_4142790775800379,False,130.0,14.0,"📈 Hôm nay, ngày 11 tháng 7, là Ngày Dân số Th...",[#DYK],https://www.facebook.com/165419643537532/posts...,10.0,NaN,added_photos,2.0
2,NaN,[],NaN,2021-07-10T12:06:41+0000,https://scontent.fhan2-4.fna.fbcdn.net/v/t39.3...,NaN,165419643537532_399386351491560,False,NaN,NaN,NaN,NaN,https://www.facebook.com/events/399386351491560/,NaN,NaN,created_event,NaN
3,6.0,[hóng 中国驻胡志明市总领馆 Tổng Lãnh sự quán Trung Quốc ...,609.0,2021-07-10T01:00:41+0000,https://scontent.fhan2-3.fna.fbcdn.net/v/t39.3...,268.0,165419643537532_4139411036138353,False,17711.0,2835.0,💉✈️ 𝟐 𝐭𝐫𝐢𝐞̣̂𝐮 𝐥𝐢𝐞̂̀𝐮 𝐯𝐚̆́𝐜 𝐱𝐢𝐧 #𝐌𝐨𝐝𝐞𝐫𝐧𝐚 𝐝𝐨 𝐇𝐨𝐚...,[#𝐌𝐨𝐝𝐞𝐫𝐧𝐚],https://www.facebook.com/165419643537532/posts...,1631.0,27.0,added_photos,31.0
4,NaN,"[Mai Nga, Mi Nguyen, Nguyễn Trần Thiên Quốc, #...",16.0,2021-07-09T07:01:23+0000,https://scontent.fhan2-4.fna.fbcdn.net/v/t39.3...,1.0,165419643537532_4137169826362474,False,404.0,18.0,"📣 Vào ngày 9 tháng 7 năm 2021, các cơ quan chứ...",[],https://www.facebook.com/165419643537532/posts...,54.0,33.0,added_photos,1.0
5,NaN,"[, Well, our first priority now is when do you...",3.0,2021-07-09T04:41:18+0000,https://scontent.fhan2-3.fna.fbcdn.net/v/t39.3...,NaN,165419643537532_4137071206372336,False,126.0,16.0,"“Thông qua báo cáo này, chúng tôi kêu gọi chín...",[],https://www.facebook.com/165419643537532/posts...,2.0,NaN,added_photos,NaN
6,NaN,[Hope you enjoyed the series. If you have any ...,57.0,2021-07-09T02:31:11+0000,https://scontent.fhan2-4.fna.fbcdn.net/v/t15.5...,NaN,165419643537532_320737379595750,False,374.0,193.0,EducationUSA Pre-departure Orientation 2021,NaN,https://www.facebook.com/USConsulateHCMC/video...,37.0,1.0,added_video,1.0
7,NaN,"[, Hải Đăng anh có hứng thú không nè, Xin tài ...",36.0,2021-07-08T09:00:47+0000,https://scontent.fhan2-4.fna.fbcdn.net/v/t39.3...,7.0,165419643537532_4134586686620788,True,679.0,59.0,🎮💰 Trung tâm Hoa Kỳ tại TP.HCM xin thông báo v...,[],https://www.facebook.com/165419643537532/posts...,37.0,NaN,added_photos,13.0
8,NaN,[Thank you for your participation! Hope to see...,26.0,2021-07-08T06:57:41+0000,https://scontent.fhan2-3.fna.fbcdn.net/v/t15.5...,1.0,165419643537532_2922746074634031,False,119.0,57.0,"From Pac-Man to LOL, from Atari to PlayStation...","[#VideoGamesDay, #esports]",https://www.facebook.com/USConsulateHCMC/video...,15.0,NaN,added_video,1.0
9,NaN,[Welcome to our Virtual Pre-Departure Orientat...,78.0,2021-07-08T02:29:23+0000,https://scontent.fhan2-4.fna.fbcdn.net/v/t15.5...,2.0,165419643537532_543046383408530,True,374.0,170.0,Viên chức Lãnh sự đang trình bày những quy địn...,NaN,https://www.facebook.com/USConsulateHCMC/video...,76.0,1.0,added_video,NaN


In [348]:
df_out = df_out.apply(get_comments, axis=1)

KeyError: 'photo view'

In [352]:
df_out

,comments,comments_count,created_time,full_picture,id,is_popular,message,message_tags,permalink_url,shares,status_type
0,"[U.S.A NUMBER ONE!, wow wow wow 🤩 , Đứng nghiê...",7.0,2021-07-11T12:33:27+0000,https://scontent.fhan2-2.fna.fbcdn.net/v/t39.3...,165419643537532_4142798599132930,False,🇺🇸🏀 #Bạncóbiết Đội tuyển bóng rổ nam và nữ quố...,"[#Bạncóbiết, #DYK]",https://www.facebook.com/165419643537532/posts...,3.0,added_photos
1,"[, We need more vigara...I think it alway work...",2.0,2021-07-11T09:29:49+0000,https://scontent.fhan2-2.fna.fbcdn.net/v/t39.3...,165419643537532_4142790775800379,False,"📈 Hôm nay, ngày 11 tháng 7, là Ngày Dân số Th...",[#DYK],https://www.facebook.com/165419643537532/posts...,10.0,added_photos
2,[],NaN,2021-07-10T12:06:41+0000,https://scontent.fhan2-4.fna.fbcdn.net/v/t39.3...,165419643537532_399386351491560,False,NaN,NaN,https://www.facebook.com/events/399386351491560/,NaN,created_event
3,[hóng 中国驻胡志明市总领馆 Tổng Lãnh sự quán Trung Quốc ...,609.0,2021-07-10T01:00:41+0000,https://scontent.fhan2-3.fna.fbcdn.net/v/t39.3...,165419643537532_4139411036138353,False,💉✈️ 𝟐 𝐭𝐫𝐢𝐞̣̂𝐮 𝐥𝐢𝐞̂̀𝐮 𝐯𝐚̆́𝐜 𝐱𝐢𝐧 #𝐌𝐨𝐝𝐞𝐫𝐧𝐚 𝐝𝐨 𝐇𝐨𝐚...,[#𝐌𝐨𝐝𝐞𝐫𝐧𝐚],https://www.facebook.com/165419643537532/posts...,1631.0,added_photos
4,"[Mai Nga, Mi Nguyen, Nguyễn Trần Thiên Quốc, #...",16.0,2021-07-09T07:01:23+0000,https://scontent.fhan2-4.fna.fbcdn.net/v/t39.3...,165419643537532_4137169826362474,False,"📣 Vào ngày 9 tháng 7 năm 2021, các cơ quan chứ...",[],https://www.facebook.com/165419643537532/posts...,54.0,added_photos
...,...,...,...,...,...,...,...,...,...,...,...
6799,[Thnks The American Centre for holding this ev...,5.0,2011-10-03T08:53:12+0000,https://scontent.fhan2-2.fna.fbcdn.net/v/t1.18...,165419643537532_176095189136644,False,"On September 23, to mark a World Heart Day and...",NaN,https://www.facebook.com/USConsulateHCMC/photo...,NaN,added_photos
6800,"[Dear Chi Linh and all,, I was happy to join i...",7.0,2011-09-30T15:05:30+0000,https://scontent.fhan2-4.fna.fbcdn.net/v/t1.18...,165419643537532_175001792579317,False,NaN,NaN,https://www.facebook.com/USConsulateHCMC/photo...,NaN,added_photos
6801,"[Dear Chi Linh and all,, I was happy to join i...",7.0,2011-09-30T06:29:49+0000,https://scontent.fhan2-3.fna.fbcdn.net/v/t1.18...,165419643537532_174882175924612,False,NaN,NaN,https://www.facebook.com/USConsulateHCMC/photo...,NaN,added_photos
6802,[],NaN,2011-09-28T03:15:16+0000,NaN,165419643537532_174031516009678,False,The American Center - Grand Opening September ...,NaN,https://www.facebook.com/165419643537532/posts...,NaN,mobile_status_update


In [350]:
# Export to CSV
df_out.to_csv("../dataset/new_consulate_posts.csv")

# Get Videos + Livestream Data

In [9]:
## Get Videos 
allvideos = graph.get_all_connections(id='165419643537532', connection_name ='videos', fields = "id, created_time, title, description, live_status, length, is_crosspost_video, permalink_url, source")
videos = list(allvideos)
df_videos = pd.DataFrame(videos)
df_videos

,id,created_time,title,description,length,is_crosspost_video,permalink_url,source,live_status
0,212643073963295,2021-04-22T03:24:01+0000,Restore Our Earth,🥳🌳🌊 Chào mừng Ngày Trái đất 🌍! Cách đây 50 nă...,57.984,False,/USConsulateHCMC/videos/212643073963295/,https://scontent.fsgn2-1.fna.fbcdn.net/v/t66.3...,NaN
1,3970213129739105,2021-04-20T09:53:30+0000,Asians in America Through the Vietnamese Lens,What is it like to be Asian in America? What c...,7220.418,False,/USConsulateHCMC/videos/3970213129739105/,https://scontent.fsgn2-4.fna.fbcdn.net/v/t66.3...,VOD
2,2889244218015469,2021-04-19T07:42:15+0000,Ambassador Kritenbrink's Farewell,"🥳☺️❤️Cảm ơn, Việt Nam, đã là chủ nhà tuyệt vời...",208.619,False,/USConsulateHCMC/videos/2889244218015469/,https://scontent.fsgn2-2.fna.fbcdn.net/v/t66.3...,NaN
3,463745138207580,2021-04-15T07:16:04+0000,Outbreak - Epidemics in a Connected World,Hãy tham dự cùng Tổng lãnh sự Damour #CGDamour...,3379.712,False,/USConsulateHCMC/videos/463745138207580/,https://scontent.fsgn2-2.fna.fbcdn.net/v/t66.3...,VOD
4,458639862083933,2021-04-14T14:00:18+0000,ACTV Season 2 | Episode 5,Chào mừng đến với Phần 2 | Tập 5 của ACTV 📺!\n...,740.181,False,/USConsulateHCMC/videos/458639862083933/,https://scontent.fsgn2-2.fna.fbcdn.net/v/t66.3...,VOD
...,...,...,...,...,...,...,...,...,...
442,631309013615257,2014-04-29T15:14:41+0000,NaN,Cám ơn các bạn đã đến tham dự và biểu diễn rất...,212.700,False,/USConsulateHCMC/videos/631309013615257/,https://video.fsgn2-1.fna.fbcdn.net/v/t43.1792...,NaN
443,131044657068301,2013-02-19T04:13:34+0000,Cùng Vợ Chồng Ngài Tổng Lãnh Sự Mỹ Đón Tết Việt,NaN,264.270,False,/USConsulateHCMC/videos/131044657068301/,https://video.fsgn2-4.fna.fbcdn.net/v/t43.1792...,NaN
444,131043813735052,2013-02-19T04:01:47+0000,American Center's One Year Anniversary Celebra...,NaN,158.520,False,/USConsulateHCMC/videos/131043813735052/,https://video.fsgn2-1.fna.fbcdn.net/v/t42.1790...,NaN
445,128334003941429,2011-10-28T08:42:00+0000,The American Center on Vietnam Today Channel,NaN,208.820,False,/USConsulateHCMC/videos/128334003941429/,https://video.fsgn2-5.fna.fbcdn.net/v/t43.1792...,NaN


In [89]:
df_videos.to_csv("Updating_Consulate Videos.csv")

In [5]:
df_videos = pd.read_csv("Updating_Consulate Videos.csv", index_col=0)

In [6]:
df_videos

,id,created_time,title,description,live_status,length,is_crosspost_video,permalink_url,source
0,463745138207580,2021-04-15T07:16:04+0000,Outbreak - Epidemics in a Connected World,Hãy tham dự cùng Tổng lãnh sự Damour #CGDamour...,VOD,3379.712,False,/USConsulateHCMC/videos/463745138207580/,https://scontent.fsgn2-2.fna.fbcdn.net/v/t66.3...
1,458639862083933,2021-04-14T14:00:18+0000,ACTV Season 2 | Episode 5,Chào mừng đến với Phần 2 | Tập 5 của ACTV 📺!\n...,VOD,740.181,False,/USConsulateHCMC/videos/458639862083933/,https://scontent.fsgn2-2.fna.fbcdn.net/v/t66.3...
2,2815005512085524,2021-04-07T14:00:23+0000,ACTV Season 2 | Episode 4,Chào mừng đến với Phần 2 | Tập 4 của ACTV 📺!\n...,VOD,789.035,False,/USConsulateHCMC/videos/2815005512085524/,https://scontent.fsgn2-3.fna.fbcdn.net/v/t66.3...
3,929437200930676,2021-04-07T02:27:14+0000,Find the Best LL.M Program For You,Find the Best LL.M Program For You,VOD,2737.963,True,/USConsulateHCMC/videos/929437200930676/,https://scontent.fsgn2-5.fna.fbcdn.net/v/t66.3...
4,3800102960037679,2021-04-02T02:58:37+0000,Live stream: Chinh phục Học bổng Fulbright ngà...,Live stream: Chinh phục Học bổng Fulbright ngà...,VOD,5783.424,True,/USConsulateHCMC/videos/3800102960037679/,https://scontent.fsgn2-5.fna.fbcdn.net/v/t66.3...
...,...,...,...,...,...,...,...,...,...
439,631309013615257,2014-04-29T15:14:41+0000,NaN,Cám ơn các bạn đã đến tham dự và biểu diễn rất...,NaN,212.700,False,/USConsulateHCMC/videos/631309013615257/,https://video.fsgn2-1.fna.fbcdn.net/v/t43.1792...
440,131044657068301,2013-02-19T04:13:34+0000,Cùng Vợ Chồng Ngài Tổng Lãnh Sự Mỹ Đón Tết Việt,NaN,NaN,264.270,False,/USConsulateHCMC/videos/131044657068301/,https://video.fsgn2-4.fna.fbcdn.net/v/t43.1792...
441,131043813735052,2013-02-19T04:01:47+0000,American Center's One Year Anniversary Celebra...,NaN,NaN,158.520,False,/USConsulateHCMC/videos/131043813735052/,https://video.fsgn2-1.fna.fbcdn.net/v/t42.1790...
442,128334003941429,2011-10-28T08:42:00+0000,The American Center on Vietnam Today Channel,NaN,NaN,208.820,False,/USConsulateHCMC/videos/128334003941429/,https://video.fsgn2-5.fna.fbcdn.net/v/t43.1792...


In [88]:
df_videos.sort_values(by = ["length"],ascending = False)

,id,created_time,title,description,live_status,length,is_crosspost_video,permalink_url,source
336,404199617046124,2019-03-06T07:49:49+0000,CG Remark at EducationUSA Spring Fair 2019,Tổng lãnh sự Mary Tarnowka đang phát biểu khai...,LIVE_STOPPED,1209600.000,False,/USConsulateHCMC/videos/404199617046124/,NaN
419,1338159389596879,2017-05-05T07:08:54+0000,YSEALI Info Session,The YSEALI alumni are ready to share their exp...,VOD,9472.000,False,/USConsulateHCMC/videos/1338159389596879/,https://scontent.fsgn2-4.fna.fbcdn.net/v/t66.3...
207,518605188672134,2020-03-10T08:59:19+0000,English Speaking Workshop,English Speaking Workshop,VOD,8801.323,False,/USConsulateHCMC/videos/518605188672134/,https://scontent.fsgn2-4.fna.fbcdn.net/v/t66.3...
365,1815189931893820,2018-08-09T09:40:29+0000,Hành Trình Hiểu Về Nhau,Không dễ để các bậc phụ huynh thấu hiểu cho nh...,VOD,8622.548,False,/USConsulateHCMC/videos/1815189931893820/,https://scontent.fsgn2-1.fna.fbcdn.net/v/t66.3...
401,1487816741297809,2017-10-13T07:00:06+0000,Youth Scholarships Talk Show - Toạ đàm Học bổn...,Hãy tham gia cùng chúng tôi Toạ đàm Học bổng T...,VOD,7837.801,False,/USConsulateHCMC/videos/1487816741297809/,https://scontent.fsgn2-1.fna.fbcdn.net/v/t66.3...
...,...,...,...,...,...,...,...,...,...
237,730110464186112,2020-02-23T07:14:47+0000,NaN,NaN,NaN,3.550,False,/USConsulateHCMC/videos/730110464186112/,https://video.fsgn2-2.fna.fbcdn.net/v/t42.1790...
236,604838170064813,2020-02-23T08:28:01+0000,NaN,NaN,NaN,3.550,False,/USConsulateHCMC/videos/604838170064813/,https://video.fsgn2-2.fna.fbcdn.net/v/t42.1790...
235,613246212585686,2020-02-24T00:43:32+0000,NaN,NaN,NaN,3.550,False,/USConsulateHCMC/videos/613246212585686/,https://video.fsgn2-5.fna.fbcdn.net/v/t42.1790...
222,562997447634061,2020-02-26T16:56:38+0000,NaN,NaN,NaN,3.550,False,/USConsulateHCMC/videos/562997447634061/,https://video.fsgn2-4.fna.fbcdn.net/v/t42.1790...


In [76]:
live = graph.get_all_connections(id='165419643537532', connection_name ='live_videos', fields = "id, title, description, broadcast_start_time, permalink_url")
livevideos = list(live)
df_live = pd.DataFrame(livevideos)

In [77]:
df_live

,id,title,description,broadcast_start_time,permalink_url
0,3896336820445777,Outbreak - Epidemics in a Connected World,Hãy tham dự cùng Tổng lãnh sự Damour #CGDamour...,2021-04-15T08:02:51+0000,/USConsulateHCMC/videos/463745138207580/
1,3894211640658295,ACTV Season 2 | Episode 5,Chào mừng đến với Phần 2 | Tập 5 của ACTV 📺!\n...,2021-04-14T14:00:18+0000,/USConsulateHCMC/videos/458639862083933/
2,3873893012690158,ACTV Season 2 | Episode 4,Chào mừng đến với Phần 2 | Tập 4 của ACTV 📺!\n...,2021-04-07T14:00:23+0000,/USConsulateHCMC/videos/2815005512085524/
3,3872834022796057,Find the Best LL.M Program For You,Find the Best LL.M Program For You,2021-04-07T02:27:29+0000,/USConsulateHCMC/videos/929437200930676/
4,3858387320907394,Live stream: Chinh phục Học bổng Fulbright ngà...,Live stream: Chinh phục Học bổng Fulbright ngà...,2021-04-02T02:58:47+0000,/USConsulateHCMC/videos/3800102960037679/
...,...,...,...,...,...
176,1281021318644020,NaN,NaN,2017-03-07T10:24:35+0000,/USConsulateHCMC/videos/1281021315310687/
177,1280904405322378,Women in Diplomacy,Join us today with U.S. Consul General and Con...,2017-03-07T07:36:31+0000,/USConsulateHCMC/videos/1280904398655712/
178,1275606872518798,NaN,Info Session: VISA,2017-03-02T09:23:37+0000,/USConsulateHCMC/videos/1275606862518799/
179,1275585655854253,NaN,NaN,2017-03-02T08:32:27+0000,/USConsulateHCMC/videos/1275585645854254/


In [32]:
insight = graph.get_connections(id="131044657068301", connection_name = "video_insights", metric='total_video_views, total_video_views_sound_on, total_video_complete_views, total_video_reactions_by_type_total, total_video_view_time_by_age_bucket_and_gender, total_video_view_time_by_region_id, total_video_views_by_distribution_type, total_video_impressions', date_preset='yesterday', period='lifetime', show_description_from_api_doc=False)
insight

{'data': []}

In [37]:
insight.size == 0

AttributeError: 'dict' object has no attribute 'size'

In [23]:
insight["data"][6]["values"][0]["value"]

{'page_owned': 2677, 'shared': 1340}

In [29]:
insight["data"][1]

{'name': 'total_video_views_sound_on',
 'period': 'lifetime',
 'values': [{'value': 2769}],
 'title': 'Lifetime video views with sound on',
 'description': 'Lifetime: Number of times your video sound was turned on and it was viewed for 3 seconds or viewed to the end, whichever came first (total count)',
 'id': '518605188672134/video_insights/total_video_views_sound_on/lifetime'}

In [33]:
def get_insight(row):
    insight = graph.get_connections(id=row["id"], connection_name = "video_insights", metric='total_video_views, total_video_views_sound_on, total_video_complete_views, total_video_reactions_by_type_total, total_video_view_time_by_age_bucket_and_gender, total_video_view_time_by_region_id, total_video_views_by_distribution_type, total_video_impressions', date_preset='yesterday', period='lifetime', show_description_from_api_doc=False)
    if insight["data"] !=  
        row["total_views"] = insight["data"][0]["values"][0]["value"]
        row["video_views_sound_on"] = insight["data"][1]["values"][0]["value"]
        row["video_complete_views"] = insight["data"][2]["values"][0]["value"]
        if "like" in insight["data"][3]["values"][0]["value"]:
            row["like"] = insight["data"][3]["values"][0]["value"]["like"]
        if "love" in insight["data"][3]["values"][0]["value"]:
            row["love"] = insight["data"][3]["values"][0]["value"]["love"]
        if "haha" in insight["data"][3]["values"][0]["value"]:
            row["haha"] = insight["data"][3]["values"][0]["value"]["haha"]
        if "anger" in insight["data"][3]["values"][0]["value"]:
            row["anger"] = insight["data"][3]["values"][0]["value"]["anger"]
        row["video_view_time_by_age_bucket_and_gender"] = insight["data"][4]["values"][0]["value"]
        row["video_view_time_by_region_id"] = insight["data"][5]["values"][0]["value"]
        if "page_owned" in insight["data"][6]["values"][0]["value"]:
            row["video_views_by_page_owned"] = insight["data"][6]["values"][0]["value"]["page_owned"]
        if "shared" in insight["data"][6]["values"][0]["value"]:
            row["video_views_by_shared"] = insight["data"][6]["values"][0]["value"]["shared"]
        row["impression"] = insight["data"][7]["values"][0]["value"]
    return row

In [26]:
df_videos

,id,created_time,title,description,length,is_crosspost_video,permalink_url,source,live_status
0,212643073963295,2021-04-22T03:24:01+0000,Restore Our Earth,🥳🌳🌊 Chào mừng Ngày Trái đất 🌍! Cách đây 50 nă...,57.984,False,/USConsulateHCMC/videos/212643073963295/,https://scontent.fsgn2-1.fna.fbcdn.net/v/t66.3...,NaN
1,3970213129739105,2021-04-20T09:53:30+0000,Asians in America Through the Vietnamese Lens,What is it like to be Asian in America? What c...,7220.418,False,/USConsulateHCMC/videos/3970213129739105/,https://scontent.fsgn2-4.fna.fbcdn.net/v/t66.3...,VOD
2,2889244218015469,2021-04-19T07:42:15+0000,Ambassador Kritenbrink's Farewell,"🥳☺️❤️Cảm ơn, Việt Nam, đã là chủ nhà tuyệt vời...",208.619,False,/USConsulateHCMC/videos/2889244218015469/,https://scontent.fsgn2-2.fna.fbcdn.net/v/t66.3...,NaN
3,463745138207580,2021-04-15T07:16:04+0000,Outbreak - Epidemics in a Connected World,Hãy tham dự cùng Tổng lãnh sự Damour #CGDamour...,3379.712,False,/USConsulateHCMC/videos/463745138207580/,https://scontent.fsgn2-2.fna.fbcdn.net/v/t66.3...,VOD
4,458639862083933,2021-04-14T14:00:18+0000,ACTV Season 2 | Episode 5,Chào mừng đến với Phần 2 | Tập 5 của ACTV 📺!\n...,740.181,False,/USConsulateHCMC/videos/458639862083933/,https://scontent.fsgn2-2.fna.fbcdn.net/v/t66.3...,VOD
...,...,...,...,...,...,...,...,...,...
442,631309013615257,2014-04-29T15:14:41+0000,NaN,Cám ơn các bạn đã đến tham dự và biểu diễn rất...,212.700,False,/USConsulateHCMC/videos/631309013615257/,https://video.fsgn2-1.fna.fbcdn.net/v/t43.1792...,NaN
443,131044657068301,2013-02-19T04:13:34+0000,Cùng Vợ Chồng Ngài Tổng Lãnh Sự Mỹ Đón Tết Việt,NaN,264.270,False,/USConsulateHCMC/videos/131044657068301/,https://video.fsgn2-4.fna.fbcdn.net/v/t43.1792...,NaN
444,131043813735052,2013-02-19T04:01:47+0000,American Center's One Year Anniversary Celebra...,NaN,158.520,False,/USConsulateHCMC/videos/131043813735052/,https://video.fsgn2-1.fna.fbcdn.net/v/t42.1790...,NaN
445,128334003941429,2011-10-28T08:42:00+0000,The American Center on Vietnam Today Channel,NaN,208.820,False,/USConsulateHCMC/videos/128334003941429/,https://video.fsgn2-5.fna.fbcdn.net/v/t43.1792...,NaN


In [34]:
df_videos = df_videos.apply(get_insight, axis = 1)

TypeError: argument of type 'int' is not iterable